In [1]:
from tensorflow import keras
import numpy as np
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.cluster import KMeans

In [2]:
# Needed to downgrade scikit-learn go get the LinearRegressor wrapper class to work
! pip install scikit-learn==0.21.2

In [3]:
from google.colab import files
uploaded = files.upload()

Saving housing.csv to housing (1).csv


In [4]:
# CHAS is categorical
# RAD might be categorical
housing = pd.read_table('housing.csv', header=None)
housing = pd.DataFrame(housing[0].str.split().to_list(),
                       columns=['CRIM','ZN','INDUS','CHAS','NOX',
                                'RM','AGE','DIS','RAD','TAX',
                                'PTRATIO','MEDV','B','LSTAT'])
housing = housing.astype('float64')
housing.CHAS = pd.Categorical(housing.CHAS)
housing.RAD = pd.Categorical(housing.RAD)
housing

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,MEDV,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48,22.0


In [5]:
# separate features and labels
features = housing.copy()
medv = features.pop('MEDV')

In [6]:
# build dataset
dataset = tf.data.Dataset.from_tensor_slices((features.values, medv.values))

In [7]:
# check top entries
for i,j in dataset.take(2):
  print(f'Features: {i}\nTarget: {j}\n')

Features: [6.320e-03 1.800e+01 2.310e+00 0.000e+00 5.380e-01 6.575e+00 6.520e+01
 4.090e+00 1.000e+00 2.960e+02 1.530e+01 4.980e+00 2.400e+01]
Target: 396.9

Features: [2.7310e-02 0.0000e+00 7.0700e+00 0.0000e+00 4.6900e-01 6.4210e+00
 7.8900e+01 4.9671e+00 2.0000e+00 2.4200e+02 1.7800e+01 9.1400e+00
 2.1600e+01]
Target: 396.9



In [8]:
# split into train and test
# batch size 64
n_data = len(dataset)
n_train = int(0.9*n_data)
batch_size = 1

tf.random.set_seed(49)
dataset_shuff = dataset.shuffle(buffer_size=n_data, reshuffle_each_iteration=False)
train_data = dataset_shuff.take(n_train).batch(batch_size)
test_data = dataset_shuff.skip(n_train).batch(batch_size)

In [9]:
# verify shape
for feat, lab in test_data.take(2):
  print(lab)

tf.Tensor([396.21], shape=(1,), dtype=float64)
tf.Tensor([396.9], shape=(1,), dtype=float64)


In [10]:
for feat, lab in train_data.take(2):
  print(lab)

tf.Tensor([396.9], shape=(1,), dtype=float64)
tf.Tensor([391.71], shape=(1,), dtype=float64)


In [11]:
# make X_train, y_train
X_train = np.empty((1, 13))
y_train = np.empty((1,))
count = 0
for i, j in train_data:
  count += 1
  X_train = np.append(X_train, i.numpy(), axis=0)
  y_train = np.append(y_train, j.numpy(), axis=0)
print(count)

455


In [12]:
# make X_test, y_test
X_test = np.empty((1, 13))
y_test = np.empty((1,))
count = 0
for i, j in test_data: 
  count += 1
  X_test = np.append(X_test, i.numpy(), axis=0)
  y_test = np.append(y_test, j.numpy(), axis=0)
print(count)

51


In [13]:
X_train.shape

(456, 13)

In [14]:
y_train.shape

(456,)

In [15]:
X_test.shape

(52, 13)

In [16]:
y_test.shape

(52,)

## Randomized Search CV

In [17]:
# Needed to downgrade scikit-learn go get the LinearRegressor wrapper class to work
! pip install scikit-learn==0.21.2

In [18]:
from tensorflow import keras
import numpy as np

# Your code here

np.random.seed(42)
tf.random.set_seed(42)
from keras.layers.normalization import BatchNormalization

keras.backend.clear_session()

def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[13,]):
    model = keras.models.Sequential()
    model.add(BatchNormalization())
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="elu"))
    model.add(keras.layers.Dense(1, activation="linear"))
    optimizer = keras.optimizers.Adam(lr=learning_rate)
    model.compile(
        optimizer=optimizer,
        loss=tf.losses.MeanSquaredError(),
        metrics=[tf.metrics.RootMeanSquaredError()]
    )
    return model

In [19]:
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)

In [20]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV
 
param_distribs = {
    "n_hidden": [1, 2, 3, 4, 5],
    "n_neurons": np.arange(1, 100),
    "learning_rate": reciprocal(3e-4, 3e-2),
}

rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10, cv=3, verbose=2)
rnd_search_cv.fit(X_train, y_train, epochs=200, 
                  validation_data = (X_test, y_test), 
                  callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] learning_rate=0.001683454924600351, n_hidden=5, n_neurons=15 ....
Epoch 1/200


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


10/10 [==============================] - 1s 61ms/step - loss: 131076.4332 - root_mean_squared_error: 362.0337 - val_loss: 138898.2500 - val_root_mean_squared_error: 372.6906
Epoch 2/200
10/10 [==============================] - 0s 8ms/step - loss: 132961.1065 - root_mean_squared_error: 364.6108 - val_loss: 136985.4375 - val_root_mean_squared_error: 370.1154
Epoch 3/200
10/10 [==============================] - 0s 8ms/step - loss: 130464.4602 - root_mean_squared_error: 361.1839 - val_loss: 135132.1094 - val_root_mean_squared_error: 367.6032
Epoch 4/200
10/10 [==============================] - 0s 8ms/step - loss: 128073.6151 - root_mean_squared_error: 357.8469 - val_loss: 132540.0469 - val_root_mean_squared_error: 364.0605
Epoch 5/200
10/10 [==============================] - 0s 7ms/step - loss: 123213.8487 - root_mean_squared_error: 351.0115 - val_loss: 128658.3438 - val_root_mean_squared_error: 358.6898
Epoch 6/200
10/10 [==============================] - 0s 9ms/step - loss: 116982.8395 -

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    6.1s remaining:    0.0s


10/10 [==============================] - 1s 27ms/step - loss: 132054.5007 - root_mean_squared_error: 363.3902 - val_loss: 136982.4844 - val_root_mean_squared_error: 370.1115
Epoch 2/200
10/10 [==============================] - 0s 7ms/step - loss: 132383.0398 - root_mean_squared_error: 363.8360 - val_loss: 133673.7812 - val_root_mean_squared_error: 365.6143
Epoch 3/200
10/10 [==============================] - 0s 8ms/step - loss: 131613.4226 - root_mean_squared_error: 362.7816 - val_loss: 129621.4922 - val_root_mean_squared_error: 360.0298
Epoch 4/200
10/10 [==============================] - 0s 7ms/step - loss: 126096.2926 - root_mean_squared_error: 355.0856 - val_loss: 122930.4609 - val_root_mean_squared_error: 350.6144
Epoch 5/200
10/10 [==============================] - 0s 7ms/step - loss: 119987.1804 - root_mean_squared_error: 346.3835 - val_loss: 110959.7109 - val_root_mean_squared_error: 333.1062
Epoch 6/200
10/10 [==============================] - 0s 8ms/step - loss: 111765.7784 -

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:  3.9min finished


15/15 [==============================] - 1s 19ms/step - loss: 129321.1826 - root_mean_squared_error: 359.5737 - val_loss: 196525.0938 - val_root_mean_squared_error: 443.3115
Epoch 2/200
15/15 [==============================] - 0s 6ms/step - loss: 66555.9448 - root_mean_squared_error: 257.5257 - val_loss: 182773.6094 - val_root_mean_squared_error: 427.5203
Epoch 3/200
15/15 [==============================] - 0s 6ms/step - loss: 26869.8243 - root_mean_squared_error: 163.7926 - val_loss: 136571.2031 - val_root_mean_squared_error: 369.5554
Epoch 4/200
15/15 [==============================] - 0s 6ms/step - loss: 13129.7493 - root_mean_squared_error: 114.4310 - val_loss: 39233.3516 - val_root_mean_squared_error: 198.0741
Epoch 5/200
15/15 [==============================] - 0s 5ms/step - loss: 7988.8183 - root_mean_squared_error: 89.3490 - val_loss: 17493.5508 - val_root_mean_squared_error: 132.2632
Epoch 6/200
15/15 [==============================] - 0s 5ms/step - loss: 6606.4986 - root_mean

RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasRegressor object at 0x7f83a15d1ad0>,
                   iid='warn', n_iter=10, n_jobs=None,
                   param_distributions={'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f83a15d3f50>,
                                        'n_hidden': [1, 2, 3, 4, 5],
                                        'n_neurons': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 1...
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
       86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_tra

In [21]:
rnd_search_cv.best_params_

{'learning_rate': 0.028943055227160282, 'n_hidden': 1, 'n_neurons': 76}

In [22]:
# trained model 
model = rnd_search_cv.best_estimator_.model

In [23]:
model.evaluate(X_test, y_test)

2/2 [==============================] - 0s 10ms/step - loss: 4214.7661 - root_mean_squared_error: 64.9212


[4214.76611328125, 64.92122650146484]

In [24]:
model.evaluate(X_test, y_test)

2/2 [==============================] - 0s 3ms/step - loss: 4214.7661 - root_mean_squared_error: 64.9212


[4214.76611328125, 64.92122650146484]